In [246]:
import pandas as pd
import numpy as np

In [247]:
df_train=pd.read_csv('../../Datasets/train.csv')
df_test=pd.read_csv('../../Datasets/test.csv')

In [248]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [249]:
df_train=df_train.drop(['PassengerId'],axis=1)

In [250]:
df_train['Age'].fillna(df_train['Age'].mean(),inplace=True)  #fill missing values with average ages

In [251]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       891 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


In [252]:
df_train['Sex']=df_train['Sex'].apply(lambda X:1 if X=='male'else 0)

In [253]:
df_train['Embarked'].fillna(df_train['Embarked'].mode()[0],inplace=True)
#filled empty rows with S having highest probability

In [254]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(sparse=False)
ohe.fit_transform(df_train[['Embarked']])
embark_enc=ohe.fit_transform(df_train[['Embarked']])

In [255]:
ohe.categories_

[array(['C', 'Q', 'S'], dtype=object)]

In [256]:
embark_enc_df=pd.DataFrame(data=embark_enc,columns=ohe.categories_[0],dtype=int)

In [257]:
X=df_train[['Pclass','Age','SibSp','Parch','Fare','Sex']]
X=pd.concat([X,embark_enc_df],axis=1)
y=df_train['Survived']
X.head()

,Pclass,Age,SibSp,Parch,Fare,Sex,C,Q,S
0,3,22.0,1,0,7.2500,1,0,0,1
1,1,38.0,1,0,71.2833,0,1,0,0
2,3,26.0,0,0,7.9250,0,0,0,1
3,1,35.0,1,0,53.1000,0,0,0,1
4,3,35.0,0,0,8.0500,1,0,0,1


In [258]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=11)
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

param_grid = { "criterion" : ["gini", "entropy"], "min_samples_leaf" : [1, 5, 10, 25, 50, 70], "min_samples_split" : [2, 4, 10, 12, 16, 18, 25, 35], "n_estimators": [100, 400, 700, 1000, 1500]}
from sklearn.model_selection import GridSearchCV, cross_val_score
rf = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=1, n_jobs=-1)
clf = GridSearchCV(estimator=rf, param_grid=param_grid, n_jobs=-1)
clf.fit(X_train, y_train)
clf.bestparams

In [259]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier


In [260]:
model=RandomForestClassifier(criterion = "gini",min_samples_leaf = 5,min_samples_split = 5,n_estimators=100,max_features='auto',oob_score=True,random_state=55,n_jobs=-1)
model.fit(X_train,y_train)
model.score(X_train,y_train),model.score(X_test,y_test)

(0.8665730337078652, 0.8770949720670391)

In [261]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [262]:
df_test['Age'].fillna(df_train['Age'].mean(),inplace=True)
df_test['Fare'].fillna(df_train['Fare'].mean(),inplace=True)

In [263]:
df_test['Sex'] = df_test['Sex'].apply(lambda X : 1 if X=='male' else 0)

In [264]:
embark_enc=ohe.transform(df_test[['Embarked']])
embark_enc_df=pd.DataFrame(data=embark_enc,columns=ohe.categories_[0],dtype=int)
X=df_test[['Pclass','Age','SibSp','Parch','Fare','Sex']]
X=pd.concat([X,embark_enc_df],axis=1)

In [265]:
yp=model.predict(X)

In [266]:
df_test['Survived']=yp

In [267]:
df_test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'Survived'],
      dtype='object')

In [268]:
df_test[['PassengerId','Survived']].to_csv('sub6.csv', index=False)